In [2]:
# coding:utf8
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from tensorflow.python.framework import ops

%matplotlib auto

ops.reset_default_graph() 

def add_layer(inputs,in_size,out_size,n_layer,activation_function=None):
    layer_name = 'layer%s'% n_layer
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size,out_size]),name='W') # 就是建造一个数组，[行,列]
            tf.summary.histogram(layer_name+'/weights',Weights) # 让weights
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1,out_size]) + 0.1,name='b')
            tf.summary.histogram(layer_name+'/biases',biases)
        with tf.name_scope('Wx'):
            Wx_plus_b=tf.matmul(inputs,Weights) + biases # 计算
        if activation_function is None: # 如果不指定激励函数
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b) # 如果指定了激励函数
        tf.summary.histogram(layer_name+'/outputs',outputs)
        return outputs

#读取数据
npg= pd.read_csv('nanoindentation.csv',header=0)
npg_df = npg.values
npg_arr= np.array(npg_df)
scarler= preprocessing.MinMaxScaler()
npg_arr_minmax=scarler.fit_transform(npg_arr)
x= npg_arr_minmax[:,:2]
y= npg_arr_minmax[:,2:3]

#交叉验证
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=2)
costs = [] 

with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32,[None,2],name='x_input')
    ys = tf.placeholder(tf.float32,[None,1],name='y_input')

l1 = add_layer(xs,2,8,n_layer=1,activation_function=tf.nn.relu) # 输入数据是xs,１个输入，8个输出，层数是１，激励函数是relu
predition = add_layer(l1,8,1,n_layer=2,activation_function=None) # 输入数据是l1的输出, 8个输入，１个输出，层数是２，无激励函数

with tf.name_scope('loss'):
    loss = tf.losses.mean_squared_error(ys,predition)
    tf.summary.scalar('loss',loss) # 记录loss的，可以用tensorboard查看

learning_rate=0.2

with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
saver = tf.train.Saver()
 
#初始化权重和阈值
init = tf.global_variables_initializer()
sess = tf.Session()
summary_op = tf.summary.merge_all()
writer = tf.summary.FileWriter("npg1/",sess.graph)
sess.run(init)

fig = plt.figure() # 用这个画图
ax = fig.add_subplot(1,1,1) # 图片编号
num=np.arange(1,20).reshape(y_train.shape)
ax.scatter(num,y_train) # 点的编号
plt.xticks(num)
plt.ion() # 不用停顿
plt.show()

# 训练部分
print("训练开始...")
for epoch in range(5000):
    #为了将loss画图用了一种莫名奇妙的方法
    epoch_cost = 0
    _,cost = sess.run([optimizer,loss],feed_dict={xs:x_train,ys:y_train})
    epoch_cost=cost
    if epoch%100 == 0:
        print("第%3d次训练的损失值：%lf"% (epoch,epoch_cost))
        result = sess.run(summary_op,feed_dict={xs:x_train,ys:y_train})
        writer.add_summary(result,epoch) # 每i步输出一个结果
        try:
            ax.lines.remove(lines[0]) # 去除上次的线
        except Exception:
            pass
        
        predition_value = sess.run(predition,feed_dict={xs:x_train}) # 预测值
        lines = ax.plot(num,predition_value,'r-',lw=2) # 用红色，宽度为５的曲线的形式画出来预测曲线
        plt.pause(0.1)
        #不知道要不要加
        plt.show()

    if epoch%5 == 0:
        costs.append(epoch_cost)
        
saver.save(sess, 'test-ckpt/model')

print("训练集均方误差：",sess.run(loss, feed_dict={xs: x_train, ys: y_train}))


# plot the loss
#plt.plot(np.squeeze(costs))
#plt.ylabel('costs')
#plt.xlabel('epochs (per 5)')
#plt.title("Learning rate = " + str(learning_rate))
#plt.show()

# 测试部分
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
print_tensors_in_checkpoint_file("test-ckpt/model",'weights,biases',None, True)

with tf.name_scope('inputs'):
    xs2 = tf.placeholder(tf.float32,[None,2],name='x_input')
    ys2 = tf.placeholder(tf.float32,[None,1],name='y_input')
with tf.name_scope('parameter'):
    W1 = tf.Variable(tf.random_normal([2,8]),name='W1')
    W2 = tf.Variable(tf.random_normal([8,1]),name='W2')
    b1 = tf.Variable(tf.random_normal([1,8]),name='b1')
    b2 = tf.Variable(tf.random_normal([1,1]),name='b2')

# 把保存的变量提出来赋值给相应的变量
saver = tf.train.Saver({"layer1/weights/W":W1, "layer2/weights/W":W2, "layer1/biases/b":b1, "layer2/biases/b":b2})

a1 = tf.nn.relu(tf.matmul(xs2,W1)+b1)
a2 = tf.matmul(a1,W2)+b2

mse = tf.losses.mean_squared_error(a2,ys2)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    saver.restore(sess, 'test-ckpt/model')
    #print(sess.run(W1), sess.run(W2))
    print("测试集均方误差：",sess.run(mse, feed_dict={xs2: x_test, ys2: y_test}))
    a2_value=sess.run(a2,feed_dict={xs2: x_test, ys2: y_test})
    #a2_value_train=sess.run(a2,feed_dict={xs2: x_train, ys2: y_train})
    #print(a2_value)

Using matplotlib backend: Qt5Agg
训练开始...
第  0次训练的损失值：0.152008
第100次训练的损失值：0.004220
第200次训练的损失值：0.003853
第300次训练的损失值：0.003681
第400次训练的损失值：0.003582
第500次训练的损失值：0.003520
第600次训练的损失值：0.003440
第700次训练的损失值：0.003361
第800次训练的损失值：0.003304
第900次训练的损失值：0.003265
第1000次训练的损失值：0.003236
第1100次训练的损失值：0.003215
第1200次训练的损失值：0.003062
第1300次训练的损失值：0.002786
第1400次训练的损失值：0.002616
第1500次训练的损失值：0.002486
第1600次训练的损失值：0.002375
第1700次训练的损失值：0.002280
第1800次训练的损失值：0.002198
第1900次训练的损失值：0.002127
第2000次训练的损失值：0.002069
第2100次训练的损失值：0.002023
第2200次训练的损失值：0.001981
第2300次训练的损失值：0.001942
第2400次训练的损失值：0.001906
第2500次训练的损失值：0.001872
第2600次训练的损失值：0.001839
第2700次训练的损失值：0.001809
第2800次训练的损失值：0.001779
第2900次训练的损失值：0.001752
第3000次训练的损失值：0.001725
第3100次训练的损失值：0.001700
第3200次训练的损失值：0.001675
第3300次训练的损失值：0.001652
第3400次训练的损失值：0.001629
第3500次训练的损失值：0.001606
第3600次训练的损失值：0.001583
第3700次训练的损失值：0.001560
第3800次训练的损失值：0.001539
第3900次训练的损失值：0.001517
第4000次训练的损失值：0.001497
第4100次训练的损失值：0.001476
第4200次训练的损失值：0.001457
第4300次训练的损失值：0.001437
第

In [8]:
d=scarler.inverse_transform(np.append(x_test,a2_value,axis=1))
print(d,'\n')
#a=npg_predicted_ori[:,0].reshape((14,1))
#b=npg_predicted_ori[:,1].reshape((14,1))
#c=npg_predicted_ori[:,2].reshape((14,1))
#d=np.concatenate((a,b,c),axis=1)
#print(d)
e=scarler.inverse_transform(np.append(x_test,y_test,axis=1))
print(e,'\n')
print((e-d)/e,'\n')
print(np.mean(((e-d)/e),axis=0))
%matplotlib auto
plt.plot(d[:,2])
plt.plot(e[:,2])
plt.xticks(np.arange(1,16,1))

[[  0.35        18.9         94.18530427]
 [  0.27        43.          11.5818106 ]
 [  0.28        43.          13.16405284]
 [  0.33        33.4         24.54144933]
 [  0.3         50.          16.84330406]
 [  0.29        43.          14.74629449]
 [  0.3         43.          16.32853498]
 [  0.33        58.7         22.22981243]
 [  0.33        19.1         71.03018421]
 [  0.34        43.          22.65750277]
 [  0.31        43.          17.91077839]
 [  0.28        43.          13.16405284]
 [  0.33        65.3         22.71516604]
 [  0.27        41.          11.57933542]] 

[[  0.35        18.9         86.79      ]
 [  0.27        43.           8.35106383]
 [  0.28        43.          11.06382979]
 [  0.33        33.4         25.3       ]
 [  0.3         50.          15.39453125]
 [  0.29        43.          13.77659574]
 [  0.3         43.          14.375     ]
 [  0.33        58.7         25.8       ]
 [  0.33        19.1         64.15      ]
 [  0.34        43.          27

([<matplotlib.axis.XTick at 0x29b5860c278>,
 <a list of 15 Text xticklabel objects>)

In [5]:
fig = plt.figure() # 用这个画图
ax = fig.add_subplot(1,1,1) # 图片编号
num=np.arange(1,20).reshape(y_train.shape)
ax.scatter(num,y_train) # 点的编号
lines = ax.plot(num,predition_value,'r-',lw=2)
plt.show()

In [9]:
%matplotlib auto
plt.plot(np.squeeze(costs))
plt.ylabel('costs')
plt.xlabel('epochs (per 5)')
#plt.xticks(np.arange(0,5000,100))
plt.title("Learning rate = " + str(learning_rate))
plt.show()

Using matplotlib backend: Qt5Agg
